# Event Bank

The EventBank class is used to interact with a local directory of event files. It implements (mostly) the same interfaces as the FDSN client in obspy, plus adds some additional methods. Unlike the [WaveBank](wavebank.pynb), which uses HDF5 to index waveforms, EventBank uses a [SQLite](https://www.sqlite.org/index.html) which is more suitable for frequent updates.

## Quickstart

In [1]:
import obspy
import obsplus
from obsplus.events.utils import catalog_to_directory

# get an event fetcher from a copy of the Crandall dataset
crandall = obsplus.copy_dataset('crandall')

# path to directory where events are stored
event_path = crandall.event_path

# Init the event bank
bank = obsplus.EventBank(event_path)

# ensure index is up-to-date
bank.update_index() 

### Accessing the index 
The index can be accessed directly to get a summary of the events contained in the archive. Depending on the task, it can be more natural to work with the index dataframe than the obspy catalog objects directly.

In [2]:
bank.read_index()

,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,horizontal_uncertainty,local_magnitude,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,path
event_id,,,,,,,,,,,,,,,,,,,,,
smi:local/248828,1.186365e+09,39.4617,-111.2378,6570.0,1.78,LR,-999,None,None,1.78,...,0.8936,-999,23,None,1.545330e+09,DC,NIOSH,1.539203e+09,None,/2007/08/06/2007-08-06T01-44-48-48828.xml
smi:local/248843,1.186397e+09,39.4615,-111.2317,2050.0,1.57,LR,-999,None,None,1.57,...,0.8237,-999,29,None,1.545330e+09,DC,NIOSH,1.539204e+09,None,/2007/08/06/2007-08-06T10-47-25-48843.xml
smi:local/248839,1.186390e+09,39.4635,-111.2277,410.0,4.37,LR,-999,None,None,4.37,...,1.7356,-999,134,None,1.545330e+09,DC,NIOSH,1.539204e+09,None,/2007/08/06/2007-08-06T08-48-40-48839.xml
smi:local/248883,1.186453e+09,39.4632,-111.2230,4180.0,1.26,None,-999,None,None,1.26,...,0.8834,-999,14,None,1.545330e+09,DC,NIOSH,1.539206e+09,None,/2007/08/07/2007-08-07T02-14-24-48883.xml
smi:local/248887,1.186458e+09,39.4625,-111.2152,4160.0,1.45,None,-999,None,None,1.45,...,0.5716,-999,15,None,1.545330e+09,DC,NIOSH,1.539206e+09,None,/2007/08/07/2007-08-07T03-44-18-48887.xml
smi:local/248882,1.186452e+09,39.4648,-111.2255,1790.0,2.08,None,-999,None,None,2.08,...,0.9935,-999,35,None,1.545330e+09,DC,NIOSH,1.539206e+09,None,/2007/08/07/2007-08-07T02-05-04-48882.xml
smi:local/248925,1.186523e+09,39.4627,-111.2200,4620.0,1.65,None,-999,None,None,1.65,...,0.5704,-999,19,None,1.545330e+09,DC,NIOSH,1.539296e+09,None,/2007/08/07/2007-08-07T21-42-51-48925.xml
smi:local/248891,1.186471e+09,39.4605,-111.2242,3240.0,2.24,None,-999,None,None,2.24,...,0.9901,-999,35,None,1.545330e+09,None,None,NaN,None,/2007/08/07/2007-08-07T07-13-05-48891.xml


The index contains the following columns:

In [3]:
print(bank.read_index().columns)

Index(['time', 'latitude', 'longitude', 'depth', 'magnitude',
       'event_description', 'associated_phase_count', 'azimuthal_gap',
       'horizontal_uncertainty', 'local_magnitude', 'moment_magnitude',
       'duration_magnitude', 'magnitude_type', 'p_phase_count',
       's_phase_count', 'p_pick_count', 's_pick_count', 'standard_error',
       'used_phase_count', 'station_count', 'vertical_uncertainty', 'updated',
       'author', 'agency_id', 'creation_time', 'version', 'path'],
      dtype='object')


### Get events
The EventBank can, of course, be used to get obspy event objects based on query parameters.

In [4]:
catalog = bank.get_events(minmagnitude=2)

In [5]:
print(catalog)

3 Event(s) in Catalog:
2007-08-06T08:48:40.010000Z | +39.464, -111.228 | 4.2 mb
2007-08-07T02:05:04.490000Z | +39.465, -111.225 | 2.44 ml
2007-08-07T07:13:05.760000Z | +39.461, -111.224 | 2.55 ml


## Put events
Events can also be saved to disk using the `put_events` method.

In [6]:
print(f'The bank has {len(bank.read_index())} events before put_events call.')

The bank has 8 events before put_events call.


In [7]:
bank.put_events(obspy.read_events())
print(f'The bank has {len(bank.read_index())} events after the put_events call.')

The bank has 11 events after the put_events call.
